# Multiple flows

1. Flow 1 runs, inputs bar1 and outputs bar2
2. Flow 2 runs, inputs Flow1.input, Flow1.output

In [13]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

import pandas as pd
from doltpy.core import Dolt
from doltpy.core.write import import_df

dolt = Dolt.init(".")

df_v1 = pd.DataFrame({"A": [1,1,1], "B": [1,1,1]})
df_v2 = pd.DataFrame({"A": [1,1,1,2,2,2], "B": [1,1,1,2,2,2]})

import_df(dolt, "bar", df_v1.reset_index(), ["index"], "create")
dolt.add("bar")
dolt.commit("Initialize bar")

v1 = list(dolt.log(number="1").keys())[0]

import_df(dolt, "bar", df_v2.reset_index(), ["index"], "update")
dolt.add("bar")
dolt.commit("Add rows to bar")

v2 = list(dolt.log(number="1").keys())[0]

In [15]:
!poetry run python3 demo_one.py run

Metaflow 2.2.5.post25+gitaad3052 executing MultiFlowDemo1 for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 21:50:15.338 Workflow starting (run-id 1610949015330315):
2021-01-17 21:50:15.344 [1610949015330315/start/1 (pid 32857)] Task is starting.
2021-01-17 21:50:16.412 [1610949015330315/start/1 (pid 32857)] 01-17 21:50:16 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 21:50:16.440 [1610949015330315/start/1 (pid 32857)] 01-17 21:50:16 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 21:50:16.499 [1610949015330315/start/1 (pid 32857)] 01-17 21:50:16 doltpy.core.dolt INFO     * master                                        	usku3to1i48gkql3osi4g32gkcflgb6r
2021-01-17 21:50:16.661 [1610949015330315/start/1 (pid 32857)] 
2021-01-17 21:50:16.661 [1610949015330315/start/1 (pid 32857)] 01-17 21:50:16 dolt

2021-01-17 21:50:21.545 [1610949015330315/end/3 (pid 32932)] 01-17 21:50:21 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
2021-01-17 21:50:21.552 [1610949015330315/end/3 (pid 32932)] 01-17 21:50:21 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-17 21:50:21.713 [1610949015330315/end/3 (pid 32932)] Task finished successfully.
2021-01-17 21:50:21.713 Done!
01-17 21:50:21 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-17 21:50:21 doltpy.core.system_helpers INFO     No processes to clean up, exiting


In [19]:
!poetry run python3 demo_two.py run --flow-dep MultiFlowDemo1/1610949015330315

Metaflow 2.2.5.post25+gitaad3052 executing MultiFlowDemo2 for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 21:53:45.895 Workflow starting (run-id 1610949225887511):
2021-01-17 21:53:45.902 [1610949225887511/start/1 (pid 32993)] Task is starting.
2021-01-17 21:53:46.974 [1610949225887511/start/1 (pid 32993)] 01-17 21:53:46 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 21:53:47.063 [1610949225887511/start/1 (pid 32993)] 01-17 21:53:47 doltpy.core.dolt INFO     flow_name,run_id,step_name,task_id,kind,database,table_name,commit,timestamp
2021-01-17 21:53:47.069 [1610949225887511/start/1 (pid 32993)] MultiFlowDemo1,1610949015330315,start,1,read,.,bar,usku3to1i48gkql3osi4g32gkcflgb6r,1.610949e+09
2021-01-17 21:53:47.069 [1610949225887511/start/1 (pid 32993)] 
2021-01-17 21:53:47.069 [1610949225887511/start/1 (pid 32993)] 01-17 21:53:47 doltpy.core.dolt INFO   

2021-01-17 21:53:49.193 [1610949225887511/middle/2 (pid 33045)] 01-17 21:53:49 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 21:53:49.218 [1610949225887511/middle/2 (pid 33045)] 01-17 21:53:49 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 21:53:49.275 [1610949225887511/middle/2 (pid 33045)] 01-17 21:53:49 doltpy.core.dolt INFO     * master                                        	kl3vpdq0qpbmjbk2n9illdueef9u9ro6
2021-01-17 21:53:49.374 [1610949225887511/middle/2 (pid 33045)] 
2021-01-17 21:53:49.374 [1610949225887511/middle/2 (pid 33045)] 01-17 21:53:49 doltpy.core.write.write INFO     No import mode specified, table exists, using "update"
2021-01-17 21:53:49.457 [1610949225887511/middle/2 (pid 33045)] 01-17 21:53:49 doltpy.core.write.write INFO     Importing to table baz in dolt directory located in ., import mode update
2021-01-17 21:53:49.457 [1610949225887511/m

In [20]:
!dolt sql -q "SELECT * from metadata;"

+----------------+------------------+-----------+---------+-------+----------+------------+----------------------------------+---------------+
| flow_name      | run_id           | step_name | task_id | kind  | database | table_name | commit                           | timestamp     |
+----------------+------------------+-----------+---------+-------+----------+------------+----------------------------------+---------------+
| MultiFlowDemo1 | 1610949015330315 | middle    | 2       | write | .        | baz        | iftmv7jctmtiqtpqlhjf4amql439flaf | 1.610949e+09  |
| MultiFlowDemo1 | 1610949015330315 | start     | 1       | read  | .        | bar        | usku3to1i48gkql3osi4g32gkcflgb6r | 1.610949e+09  |
| MultiFlowDemo2 | 1610949225887511 | middle    | 2       | write | .        | baz        | 6bsk3fsslrice4ts2d86s8te1htlthjq | 1.6109492e+09 |
| MultiFlowDemo2 | 1610949225887511 | start     | 1       | read  | .        | bar        | usku3to1i48gkql3osi4g32gkcflgb6r | 1.6109492e+09 |

In [21]:
!cat demo_two.py

import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

import pickle
import time

from metaflow import FlowSpec, step, DoltDT, Parameter
from metaflow.datatools.dolt import DoltRun
import pandas as pd
from sklearn import tree

class MultiFlowDemo2(FlowSpec):
    flow_dep = Parameter('flow-dep',  help="Specifc the tag for the input version", required=True)
    @step
    def start(self):
        flow, run = self.flow_dep.split("/")
        d = DoltRun(flow_name=flow, run_id=run)
        f_input = d.reads[0]
        f_output = d.writes[0]
        with DoltDT(run=self) as dolt:
            self.inp1 = dolt.read_table(f_input.table_name, commit=f_input.commit)
            self.inp2 = dolt.read_table(f_output.table_name, commit=f_output.commit)

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self) as dolt:

            df = self.inp1 + self.inp2

            dolt.write_table(table_name='baz', df=df, pks=['index'])

        self